<a href="https://colab.research.google.com/github/asundar0128/ResearchPapersQuery/blob/main/PubMedBiopharmaFiltering_Abhinit_Sundar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -sSL https://install.python-poetry.org | python3 -
import os
os.environ["PATH"] += ":/root/.local/bin"

Retrieving Poetry metadata

# Welcome to Poetry!

This will download and install the latest version of Poetry,
a dependency and package manager for Python.

It will add the `poetry` command to Poetry's bin directory, located at:

/root/.local/bin

You can uninstall at any time by executing this script with the --uninstall option,
and these changes will be reverted.

Installing Poetry (2.1.3)
Installing Poetry (2.1.3): Creating environment
Installing Poetry (2.1.3): Installing Poetry
Installing Poetry (2.1.3): Creating script
Installing Poetry (2.1.3): Done

Poetry (2.1.3) is installed now. Great!

To get started you need Poetry's bin directory (/root/.local/bin) in your `PATH`
environment variable.

Add `export PATH="/root/.local/bin:$PATH"` to your shell configuration file.

Alternatively, you can call Poetry explicitly with `/root/.local/bin/poetry`.

You can test that everything is set up by executing:

`poetry --version`



In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.1 MB/s eta 0:00:00


In [ ]:
!poetry new pubmed_fetcher_project --src
%cd pubmed_fetcher_project

The --src option is now the default and will be removed in a future version.
Created package pubmed_fetcher_project in pubmed_fetcher_project
/content/pubmed_fetcher_project


In [ ]:
# list of dependencies for Poetry Package
PyProjectPoetryDependency = """
[tool.poetry]
name = "pubmed-fetcher"
version = "0.1.0"
description = "Fetch PubMed papers with biotech/pharma affiliations using typed, robust, and modular Python"
authors = ["Abhinit Sundar <aks94@njit.edu>"]
packages = [{ include = "pubmed_fetcher" }]

[tool.poetry.dependencies]
python = "^3.9"
biopython = "^1.83"
pandas = "^2.2.2"
tqdm = "^4.66.4"

[tool.poetry.scripts]
get-papers-list = "pubmed_fetcher.cli:main"
"""
with open("pyproject.toml", "w") as f:
    f.write(PyProjectPoetryDependency)

In [ ]:
# creating a new directory pubmed_fetcher
import os
os.makedirs("pubmed_fetcher", exist_ok=True)
with open("pubmed_fetcher/__init__.py", "w") as f:
    f.write("")

In [ ]:
# importing packages typing, re, pandas, tqdm, and Bio
from typing import List, Dict
import re
import pandas as pd
from tqdm import tqdm
from Bio import Entrez, Medline

# Specifying my university email address for Entrez package
Entrez.email = "aks94@njit.edu"

# List of corporate keywords
corporateKeywords = [
    "pharma", "pharmaceutical", "biotech", "biotechnology",
    "inc", "ltd", "llc", "gmbh", "corp", "co.", "plc"
]

# List of institutional keywords
institutionalKeywords = [
    "university", "université", "universität", "institute", "school",
    "college", "department", "faculty", "hospital", "centre", "center",
    "clinic", "foundation", "research center", "research institute"
]

# Finding keywords for company affiliation for both corporate and institutional
def companyAffiliation(generatedAffiliation: str) -> bool:
    lowercaseAffiliation = generatedAffiliation.lower()
    return (
        any(keyword in lowercaseAffiliation for keyword in corporateKeywords) and
        not any(keyword in lowercaseAffiliation for keyword in institutionalKeywords)
    )

# Retrieving author's corresponding email address from record with each generated field data
def retrievalEmailsFromRecord(generatedRecord: Dict, debug: bool = False) -> List[str]:
    generatedEmailSet = set()
    fieldsToCheck = ["AD", "IR", "AID", "FAU"]

# checking for field data with generated record and field
    for field in fieldsToCheck:
        generatedFieldData = generatedRecord.get(field, [])
        if isinstance(generatedFieldData, str):
            generatedFieldData = [generatedFieldData]

# checking for matched emails with regex and adding emails to the email set
        for line in generatedFieldData:
            matchedEmails = re.findall(r"[\w\.-]+@[\w\.-]+\.\w+", line)
            if matchedEmails:
                for email in matchedEmails:
                    if "noreply" not in email.lower() and not email.lower().endswith("@example.com"):
                        generatedEmailSet.add(email)
                        if debug:
                            print(f"[DEBUG] Found email in field '{field}': {email}")

    return list(generatedEmailSet)

# Using esearch and read functions from Entrez within the pubmed database with a query and specified maximum results
def search_pubmed(generatedQuery: str, generatedMaximumResults: int = 200) -> List[str]:
    generatedHandle = Entrez.esearch(db="pubmed", term=generatedQuery, retmax=generatedMaximumResults)
    generatedRecord = Entrez.read(generatedHandle)
    generatedHandle.close()
    return generatedRecord["IdList"]

# Fetching the details from PubMedIDS with efetch function from Entrez for PubMed database
def fetch_details(generatedPubMedIDS: List[str]) -> List[Dict]:
    generatedHandle = Entrez.efetch(
        db="pubmed",
        id=",".join(generatedPubMedIDS),
        rettype="medline",
        retmode="text"
    )
    generatedRecords = list(Medline.parse(generatedHandle))
    generatedHandle.close()
    return generatedRecords

# Processing records with PMID, Title, Publication Date, Authors, and Affiliations
def process_records(generatedRecords: List[Dict], debug: bool = False) -> pd.DataFrame:
    generatedResults = []

# using tqdm with generatedRecords and Processing
    for generatedRecord in tqdm(generatedRecords, desc="Processing"):
        try:
          # retrieving PMID, TI, DP, FAU, and AD for the record list
            generatedPMID = generatedRecord.get("PMID", "")
            generatedTitle = generatedRecord.get("TI", "")
            generatedPublicationDateRaw = generatedRecord.get("DP", "")
            generatedAuthors = generatedRecord.get("FAU", [])
            generatedAffiliations = generatedRecord.get("AD", [])

            if isinstance(generatedAffiliations, str):
                generatedAffiliations = [generatedAffiliations]
# initializing non academic authors and company affiliations
            generatedNonAcademicAuthors = []
            generatedCompanyAffiliations = []

            for generatedAffiliation in generatedAffiliations:
                if debug:
                    print(f"[DEBUG] Affiliation: {generatedAffiliation}")

                if companyAffiliation(generatedAffiliation):
                    generatedCompanyAffiliations.append(generatedAffiliation)

            if generatedCompanyAffiliations and generatedAuthors:
                generatedNonAcademicAuthors = generatedAuthors[:len(generatedCompanyAffiliations)]

            generatedEmailList = retrievalEmailsFromRecord(generatedRecord, debug)
            generatedCorrespondingEmail = generatedEmailList[0] if generatedEmailList else ""

 # retrieving the publication year with a regex search command
            generatedPublicationYear = ""
            yearMatch = re.search(r"\b(19|20)\d{2}\b", generatedPublicationDateRaw)
            if yearMatch:
                generatedPublicationYear = yearMatch.group(0)

            if generatedCompanyAffiliations:
                generatedResults.append({
                    "PubmedID": generatedPMID,
                    "Title": generatedTitle,
                    "Publication Date": generatedPublicationYear,
                    "Non-academic Author(s)": "; ".join(generatedNonAcademicAuthors),
                    "Company Affiliation(s)": "; ".join(generatedCompanyAffiliations),
                    "Corresponding Author Email": generatedCorrespondingEmail,
                    "Raw Publication Date": generatedPublicationDateRaw
                })

# exception handling for skipping records due to error
        except Exception as generatedException:
            if debug:
                print(f"[WARN] Skipping record due to error: {generatedException}")
            continue

    return pd.DataFrame(generatedResults)

In [ ]:
# command line interface parsing logic for supporting a user-defined PubMed query string
generatedCLICode = """
import argparse
import sys
from pubmed_fetcher.core import search_pubmed, fetch_details, process_records

def main():
    generatedCommandLineParser = argparse.ArgumentParser(
        description="Fetch PubMed papers with biotech/pharma affiliations"
    )

    generatedCommandLineParser.add_argument(
        "query",
        type=str,
        help="User-defined PubMed query string (supports full PubMed syntax)"
    )

    generatedCommandLineParser.add_argument(
        "-d", "--debug",
        action="store_true",
        help="Enable debug logging during execution"
    )

    generatedCommandLineParser.add_argument(
        "-f", "--file",
        type=str,
        help="Optional CSV output filename (if omitted, prints to standard output)"
    )

    generatedParsedArguments = generatedCommandLineParser.parse_args()

    try:
        generatedPubmedIDList = search_pubmed(generatedParsedArguments.query)

        if not generatedPubmedIDList:
            print("[INFO] No results found for the provided PubMed query.")
            return

        generatedPublicationRecords = fetch_details(generatedPubmedIDList)
        generatedProcessedDataFrame = process_records(
            generatedPublicationRecords,
            debug=generatedParsedArguments.debug
        )

        if generatedProcessedDataFrame.empty:
            print("[INFO] No papers matched the biotech/pharma affiliation criteria.")
            return

        if generatedParsedArguments.file:
            generatedProcessedDataFrame.to_csv(generatedParsedArguments.file, index=False)
            print(f"[INFO] Results saved to: {generatedParsedArguments.file}")
        else:
            print(generatedProcessedDataFrame.to_csv(index=False))

    except Exception as generatedError:
        print(f"[ERROR] An unexpected error occurred: {generatedError}", file=sys.stderr)
        sys.exit(1)
"""
with open("pubmed_fetcher/cli.py", "w") as f:
    f.write(generatedCLICode)

In [ ]:
!poetry install

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: pubmed-fetcher (0.1.0)Installing the current project: pubmed-fetcher (0.1.0)


In [ ]:
!poetry run get-papers-list "(pharmaceutical OR biotech OR biotechnology OR pharma OR drug development OR biopharmaceutical OR life sciences OR medical company) AND (affiliation[Affiliation])" -f filteredPubMedArticles.csv

Processing: 100% 200/200 [00:00<00:00, 22899.05it/s]
[INFO] Results saved to: filteredPubMedArticles.csv


In [27]:
!poetry run pytest

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.3.5, pluggy-1.6.0
rootdir: /content/pubmed_fetcher_project
configfile: pyproject.toml
plugins: anyio-4.9.0, langsmith-0.4.4, typeguard-4.4.4
collected 0 items                                                              

============================ no tests ran in 0.01s =============================
